# 🚀 Ailo Colab Miner (AILO-1B)

1. Runtime > Change runtime type > T4 GPU
2. Enter wallet below
3. Runtime > Run all

In [ ]:
#@title ⚙️ 1. Setup
!pip install torch --index-url https://download.pytorch.org/whl/cu118 -q
!pip install aiohttp requests -q
import gc; gc.collect()
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"✅ GPU: {torch.cuda.get_device_name(0)} | VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f}GB")
else:
    print("❌ Enable GPU!")

In [ ]:
#@title 🔑 2. Wallet
WALLET = ""  #@param {type:"string"}
print(f"✅ {WALLET[:12]}..." if len(WALLET)>=40 else "❌ Enter wallet!")

In [ ]:
#@title 🧠 3. Model & Training
import torch, torch.nn as nn, numpy as np, requests, aiohttp, asyncio, time, base64, gzip, gc

API="https://ailo.site/api"
VER="1.4.0-colab"

class AILO1B(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb=nn.Embedding(50257,1600)
        self.pos=nn.Parameter(torch.zeros(1,512,1600))
        self.tf=nn.TransformerEncoder(nn.TransformerEncoderLayer(1600,25,6400,0.1,batch_first=True),24)
        self.out=nn.Linear(1600,50257)
    def forward(self,x):
        return self.out(self.tf(self.emb(x)*40+self.pos[:,:x.size(1)]))

class Trainer:
    def __init__(self):
        self.dev=torch.device('cuda')
        self.m=None; self.opt=None; self.steps=0; self.acc=0
    
    def init(self):
        print("🧠 Loading AILO-1B...")
        gc.collect(); torch.cuda.empty_cache()
        self.m=AILO1B().half().to(self.dev)
        print(f"   {sum(p.numel() for p in self.m.parameters()):,} params | GPU: {torch.cuda.memory_allocated()/1e9:.1f}GB")
        self.opt=torch.optim.SGD(self.m.parameters(),lr=0.001,momentum=0.9)
        self.crit=nn.CrossEntropyLoss()
        print("   ✅ Ready!")
    
    def batch(self,w):
        try:
            r=requests.get(f"{API}/cuda/training-data",params={'batchSize':1,'wallet':w},timeout=10)
            if r.ok:
                t=r.json().get('articles',[''])[0]
                tk=[ord(c)%50257 for c in t[:33]]
                tk+=[0]*(33-len(tk))
                return torch.tensor([tk[:32]]),torch.tensor([tk[1:33]])
        except:pass
        return torch.randint(0,50257,(1,32)),torch.randint(0,50257,(1,32))
    
    def step(self,x,y):
        self.m.train()
        x,y=x.to(self.dev),y.to(self.dev)
        with torch.cuda.amp.autocast():
            loss=self.crit(self.m(x).view(-1,50257),y.view(-1))/16
        loss.backward()
        self.acc+=1
        if self.acc>=16:
            nn.utils.clip_grad_norm_(self.m.parameters(),1.0)
            self.opt.step(); self.opt.zero_grad(set_to_none=True); self.acc=0
        self.steps+=1
        if self.steps%50==0: gc.collect(); torch.cuda.empty_cache()
        return loss.item()*16
    
    def grads(self):
        # Only sample gradients from output layer to save RAM!
        # Output layer is the most important for learning
        g=self.m.out.weight.grad
        if g is None: return None
        gc.collect()
        # Sample 10% of gradients to fit in RAM
        flat=g.cpu().float().flatten()
        sample=flat[::10]  # Take every 10th value
        del flat; gc.collect()
        comp=gzip.compress(sample.half().numpy().tobytes(),4)
        del sample; gc.collect()
        return base64.b64encode(comp).decode()

print("✅ Ready")

In [ ]:
#@title 🚀 4. Mine!

async def hb(w,tps):
    try:
        async with aiohttp.ClientSession() as s:
            await s.post(f"{API}/cuda/register",json={'wallet':w,'clientVersion':VER,'deviceInfo':{'gpu_name':'T4-Colab','vram_gb':15,'hashrate':tps}})
            await s.post(f"{API}/ping",json={'wallet':w,'status':'training','hashrate':f"{tps:.0f} Tok/s"})
    except:pass

async def submit(w,g,s,l):
    try:
        async with aiohttp.ClientSession() as ss:
            async with ss.post(f"{API}/cuda/submit",json={'wallet':w,'gradients':g,'epoch':s,'loss':l,'gpu':'T4-Colab'}) as r:
                if r.status==200: return (await r.json()).get('reward',0)
    except:pass
    return 0

async def run():
    if len(WALLET)<40: print("❌ Set wallet!"); return
    print("="*40+f"\n  AILO-1B Colab v{VER}\n"+"="*40)
    await hb(WALLET,0)
    t=Trainer(); t.init()
    print("\n⛏️ MINING!\n")
    rew,last_s,last_h,tps=0.0,time.time(),time.time(),0.0
    try:
        while True:
            t0=time.time()
            x,y=t.batch(WALLET)
            loss=t.step(x,y)
            tps=32/(time.time()-t0+0.001)
            if time.time()-last_h>=10: await hb(WALLET,tps); last_h=time.time()
            if t.steps%25==0: print(f"Step {t.steps} | Loss: {loss:.4f} | {tps:.0f} tok/s | GPU: {torch.cuda.memory_allocated()/1e9:.1f}GB")
            if time.time()-last_s>=300:
                print("\n📤 Submitting...")
                gc.collect(); torch.cuda.empty_cache()  # Clean before grad extraction
                g=t.grads()
                if g:
                    r=await submit(WALLET,g,t.steps,loss); rew+=r
                    print(f"💰 +{r:.4f} ALC (Total: {rew:.4f})\n")
                gc.collect()  # Clean after
                last_s=time.time()
    except KeyboardInterrupt: print(f"\n⏹️ Total: {rew:.4f} ALC")

await run()